In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_x1 = pd.read_csv('../data/train_ssvm_xy.csv')
train_x1.shape

(10000, 160)

In [3]:
train_x2 = pd.read_csv('../data/train_xy.csv')
train_x2.shape

(15000, 160)

In [4]:
test = pd.read_csv('../data/test_all.csv')
test.shape

(10000, 159)

In [5]:
train_x11 = train_x1.drop(['cust_id','cust_group','y'],axis=1)
train_x22 = train_x2.drop(['\ufeffcust_id','cust_group','y'],axis=1)
train_x = pd.concat([train_x11, train_x22])
train_x.shape

(25000, 157)

In [6]:
test_x = test.drop(['cust_id','cust_group'],axis=1)
test_x.shape

(10000, 157)

In [7]:
x = pd.concat([train_x, test_x])
x.shape

(35000, 157)

In [8]:
train_y1 = train_x1['y']
train_y2 = train_x2['y']
Y_train = train_y1.append(train_y2)
Y_train.shape

(25000,)

In [9]:
for i in range(96,158):
    col = 'x'+'_'+str(i)
    if col in x.columns.values:
        dummies_df = pd.get_dummies(x[col]).rename(columns=lambda x: col +'_'+ str(x))
        x = pd.concat([x, dummies_df], axis=1)
print(x.shape)

(35000, 364)


In [10]:
train_X = x[0:25000]
test_X = x[25000:35000]
print(train_X.shape)
print(test_X.shape)

(25000, 364)
(10000, 364)


In [11]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Reshape, Flatten, MaxPool2D
from keras.layers.convolutional import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from sklearn.cross_validation import train_test_split
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.
C:\Program Files\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
X_train,X_val,y_train,y_val= train_test_split(train_X,Y_train,test_size=0.2,random_state=2)

In [13]:
X_train = X_train.values
X_val = X_val.values
print(X_train.shape)
print(X_val.shape)

(20000, 364)
(5000, 364)


In [14]:
y_train =  y_train.values
yy_train = to_categorical(y_train)

y_val =  y_val.values
yy_val = to_categorical(y_val)

In [15]:
# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

model = Sequential()

model.add(BatchNormalization(input_shape=(364,)))
model.add(Reshape((364,1,1)))


model.add(Conv2D(filters = 16, kernel_size = 5,padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 16, kernel_size = 5,padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=2, padding='same'))
# model.add(Dropout(0.25))


model.add(Conv2D(filters = 32, kernel_size = 3,padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = 3,padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
# model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))

In [16]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])

In [17]:
history=model.fit(X_train,yy_train, batch_size=256, epochs=15, verbose=1, validation_split=0.1)

Train on 18000 samples, validate on 2000 samples
Epoch 1/15
18000/18000 [==============================] - 24s 1ms/step - loss: 0.2315 - acc: 0.9452 - val_loss: 0.3059 - val_acc: 0.9550
Epoch 2/15
18000/18000 [==============================] - 24s 1ms/step - loss: 0.1728 - acc: 0.9552 - val_loss: 0.2073 - val_acc: 0.9550
Epoch 3/15
18000/18000 [==============================] - 24s 1ms/step - loss: 0.1647 - acc: 0.9552 - val_loss: 0.1616 - val_acc: 0.9550
Epoch 4/15
18000/18000 [==============================] - 24s 1ms/step - loss: 0.1610 - acc: 0.9552 - val_loss: 0.1613 - val_acc: 0.9550
Epoch 5/15
18000/18000 [==============================] - 24s 1ms/step - loss: 0.1608 - acc: 0.9552 - val_loss: 0.1613 - val_acc: 0.9550
Epoch 6/15
18000/18000 [==============================] - 24s 1ms/step - loss: 0.1612 - acc: 0.9552 - val_loss: 0.1608 - val_acc: 0.9550
Epoch 7/15
18000/18000 [==============================] - 26s 1ms/step - loss: 0.1597 - acc: 0.9552 - val_loss: 0.1600 - val_acc:

In [18]:
predictions = model.predict_proba(X_val,verbose=1)
pre = predictions[:,1]
val_auc = metrics.roc_auc_score(y_val,pre)#验证集上的auc值
print(val_auc)

5000/5000 [==============================] - 2s 461us/step
0.793837946347


In [19]:
preds = model.predict_proba(test_X.values)
pred = preds[:,1]
pred.shape

10000/10000 [==============================] - 5s 459us/step


(10000,)

In [20]:
Submission = pd.DataFrame({'cust_id': test['cust_id'], 'pred_prob': pred})
Submission.to_csv('../result/semi_nn.csv',index=False)